In [2]:
#!/usr/bin/env python3
"""
JSON DEMO – Master JSON in Python!
Shows:
  1. Load/Dump from strings, files, APIs
  2. Pretty printing
  3. Custom class serialization
  4. Validation with schemas (jsonschema)
  5. Real API call (GitHub)
"""

import json
import requests
from datetime import datetime
from pathlib import Path

# Optional: pip install jsonschema
try:
    from jsonschema import validate, ValidationError
    JSONSCHEMA_AVAILABLE = True
except ImportError:
    JSONSCHEMA_AVAILABLE = False
    print("Warning: jsonschema not installed. Run: pip install jsonschema")

# ----------------------------------------------------------------------
# 1. Basic JSON String <-> Python Object
# ----------------------------------------------------------------------
print("1. BASIC JSON STRING ↔ PYTHON OBJECT")
json_str = '''
{
    "name": "Alice",
    "age": 30,
    "is_student": false,
    "hobbies": ["reading", "hiking", null],
    "address": {
        "street": "123 Main St",
        "city": "Seattle"
    }
}
'''

# Parse JSON → Python dict
data = json.loads(json_str)
print(f"   Parsed: {data['name']} is {data['age']} years old")

# Python → JSON string
json_output = json.dumps(data, indent=2)
print(f"   Dumped (pretty):\n{json_output}")

# ----------------------------------------------------------------------
# 2. Read/Write JSON Files
# ----------------------------------------------------------------------
print("\n2. JSON FILE I/O")
file_path = Path("user_profile.json")

# Write to file
with open(file_path, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)
print(f"   Saved to {file_path}")

# Read from file
with open(file_path, "r", encoding="utf-8") as f:
    loaded = json.load(f)
print(f"   Loaded back: {loaded['name']} lives in {loaded['address']['city']}")

# ----------------------------------------------------------------------
# 3. Custom Class ↔ JSON (using __dict__ and object_hook)
# ----------------------------------------------------------------------
print("\n3. CUSTOM CLASS SERIALIZATION")

class User:
    def __init__(self, name, age, city):
        self.name = name
        self.age = age
        self.city = city
        self.join_date = datetime.now().isoformat()

    def __repr__(self):
        return f"User({self.name}, {self.age}, {self.city})"

# Python object → JSON
user = User("Bob", 25, "Portland")
user_json = json.dumps(user.__dict__, indent=2)
print(f"   User → JSON:\n{user_json}")

# JSON → custom object
def object_hook(d):
    if 'name' in d and 'age' in d and 'city' in d:
        return User(d['name'], d['age'], d['city'])
    return d

restored_user = json.loads(user_json, object_hook=object_hook)
print(f"   JSON → User: {restored_user}")
print(f"   Join date: {restored_user.join_date}")

# ----------------------------------------------------------------------
# 4. JSON Schema Validation (optional)
# ----------------------------------------------------------------------
if JSONSCHEMA_AVAILABLE:
    print("\n4. JSON SCHEMA VALIDATION")
    schema = {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "age": {"type": "integer", "minimum": 0},
            "city": {"type": "string"}
        },
        "required": ["name", "age"]
    }

    try:
        validate(instance=user.__dict__, schema=schema)
        print("   Validation passed!")
    except ValidationError as e:
        print(f"   Validation failed: {e.message}")

# ----------------------------------------------------------------------
# 5. Real API: Fetch JSON from GitHub
# ----------------------------------------------------------------------
print("\n5. FETCH JSON FROM API (GitHub)")
try:
    response = requests.get("https://api.github.com/users/octocat")
    response.raise_for_status()
    github_data = response.json()

    print(f"   GitHub User: {github_data['login']}")
    print(f"   Name: {github_data.get('name', 'N/A')}")
    print(f"   Public Repos: {github_data['public_repos']}")
    print(f"   Bio: {github_data['bio'][:60] if github_data['bio'] else 'No bio'}...")

except Exception as e:
    print(f"   API call failed: {e}")

# ----------------------------------------------------------------------
# 6. Compact vs Pretty JSON
# ----------------------------------------------------------------------
print("\n6. COMPACT vs PRETTY")
compact = json.dumps(data, separators=(',', ':'))
pretty = json.dumps(data, indent=2, sort_keys=True)

print(f"   Compact (minified): {compact[:80]}...")
print(f"   Pretty (sorted):")
print(pretty)

# ----------------------------------------------------------------------
# Cleanup
# ----------------------------------------------------------------------
# file_path.unlink(missing_ok=True)
# print(f"\nDemo complete! Cleaned up {file_path.name}")

1. BASIC JSON STRING ↔ PYTHON OBJECT
   Parsed: Alice is 30 years old
   Dumped (pretty):
{
  "name": "Alice",
  "age": 30,
  "is_student": false,
  "hobbies": [
    "reading",
    "hiking",
    null
  ],
  "address": {
    "street": "123 Main St",
    "city": "Seattle"
  }
}

2. JSON FILE I/O
   Saved to user_profile.json
   Loaded back: Alice lives in Seattle

3. CUSTOM CLASS SERIALIZATION
   User → JSON:
{
  "name": "Bob",
  "age": 25,
  "city": "Portland",
  "join_date": "2025-12-22T19:23:02.664459"
}
   JSON → User: User(Bob, 25, Portland)
   Join date: 2025-12-22T19:23:02.664761

4. JSON SCHEMA VALIDATION
   Validation passed!

5. FETCH JSON FROM API (GitHub)
   GitHub User: octocat
   Name: The Octocat
   Public Repos: 8
   Bio: No bio...

6. COMPACT vs PRETTY
   Compact (minified): {"name":"Alice","age":30,"is_student":false,"hobbies":["reading","hiking",null],...
   Pretty (sorted):
{
  "address": {
    "city": "Seattle",
    "street": "123 Main St"
  },
  "age": 30,
  "hobbie